In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

from glob import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import models, transforms

from PIL import Image

In [ ]:
imagenet_stats = {
    "mean": (0.485, 0.456, 0.406),
    "std": (0.229, 0.224, 0.225)
}

norm_layer = transforms.Normalize(
    mean=imagenet_stats["mean"],
    std=imagenet_stats["std"]
)

resnet_transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    norm_layer
])

cifar_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    norm_layer
])

In [ ]:
# =====================================================
# CIFAR-10 DATASET WRAPPER
# =====================================================

class CIFAR10Images(Dataset):

    def __init__(self,
                 root="cifar-10-batches-py",
                 use_train=True,
                 transform=None):

        self.transform = transform
        image_buffer = []
        target_buffer = []

        if use_train:

            batch_files = [
                os.path.join(root, f"data_batch_{i}")
                for i in range(1, 6)
            ]

        else:

            batch_files = [os.path.join(root, "test_batch")]

        for file_path in batch_files:

            with open(file_path, "rb") as fh:
                payload = pickle.load(fh, encoding="bytes")

            image_buffer.append(payload[b"data"])
            target_buffer.extend(payload[b"labels"])

        raw_array = np.vstack(image_buffer).astype(np.uint8)

        # reshape → HWC layout
        self.images = raw_array.reshape(-1, 3, 32, 32) \
                               .transpose(0, 2, 3, 1)

        self.targets = target_buffer

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):

        img = Image.fromarray(self.images[index])
        label = self.targets[index]

        if self.transform is not None:
            img = self.transform(img)

        return img, label


# =====================================================
# CIFAR-10 RESNET SPLITS
# =====================================================

def build_cifar_resnet_sets(root="cifar-10-batches-py"):

    train_pool = CIFAR10Images(
        root=root,
        use_train=True,
        transform=resnet_transform
    )

    split_point = int(0.8 * len(train_pool))

    train_subset, val_subset = random_split(
        train_pool,
        [split_point, len(train_pool) - split_point]
    )

    test_subset = CIFAR10Images(
        root=root,
        use_train=False,
        transform=resnet_transform
    )

    return train_subset, val_subset, test_subset


# =====================================================
# CATS vs DOGS DATASET
# =====================================================

class CatDogImages(Dataset):

    def __init__(self, directory, transform=None):

        self.transform = transform
        self.samples = []

        for file_path in glob(os.path.join(directory, "*.jpg")):

            name = os.path.basename(file_path).lower()

            if "cat" in name:
                label = 0
            elif "dog" in name:
                label = 1
            else:
                raise ValueError(f"Invalid label source: {name}")

            self.samples.append((file_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):

        path, label = self.samples[index]

        img = Image.open(path).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        return img, label


# =====================================================
# CATS vs DOGS RESNET SPLITS
# =====================================================

def build_catdog_resnet_sets(directory="dogs-vs-cats/train"):

    dataset_pool = CatDogImages(
        directory,
        transform=resnet_transform
    )

    split_point = int(0.8 * len(dataset_pool))

    train_subset, val_subset = random_split(
        dataset_pool,
        [split_point, len(dataset_pool) - split_point]
    )

    return train_subset, val_subset


In [ ]:
class CompactClassifier(nn.Module):

    def __init__(self,
                 classes=10,
                 act_name="relu",
                 input_dims=(3, 32, 32)):

        super().__init__()

        self.activation = self._make_activation(act_name)

        in_channels = input_dims[0]

        # ----- feature extractor -----
        self.stage_a = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32)
        )

        self.stage_b = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64)
        )

        self.stage_c = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128)
        )

        self.downsample = nn.MaxPool2d(2, 2)

        flat_features = (
            128
            * (input_dims[1] // 8)
            * (input_dims[2] // 8)
        )

        # ----- classifier head -----
        self.hidden = nn.Linear(flat_features, 256)
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(256, classes)

    # -------------------------------------------------

    def _make_activation(self, name):

        lookup = {
            "relu": nn.ReLU(),
            "tanh": nn.Tanh(),
            "leaky_relu": nn.LeakyReLU()
        }

        key = name.lower()

        if key not in lookup:
            raise ValueError("Unsupported activation")

        return lookup[key]

    # -------------------------------------------------

    def _feature_pass(self, x):

        for block in (self.stage_a,
                      self.stage_b,
                      self.stage_c):

            x = self.downsample(
                self.activation(block(x))
            )

        return x

    # -------------------------------------------------

    def forward(self, x):

        x = self._feature_pass(x)

        x = torch.flatten(x, 1)

        x = self.hidden(x)
        x = self.activation(x)
        x = self.dropout(x)

        return self.output(x)

In [ ]:
# =====================================================
# RESNET-18 FACTORY
# =====================================================

def create_resnet_backbone(class_count, image_dims=(224, 224)):

    net = models.resnet18(weights=None)

    # enforce RGB input channel compatibility
    if net.conv1.in_channels != 3:
        net.conv1 = nn.Conv2d(
            3, 64,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )

    feature_width = net.fc.in_features
    net.fc = nn.Linear(feature_width, class_count)

    return net


# =====================================================
# RESNET TRAINING ROUTINE
# =====================================================

def run_resnet_fit(net,
                   dataset_tag,
                   train_stream,
                   val_stream,
                   opt,
                   loss_fn,
                   epochs=10,
                   device="cpu"):

    net.to(device)

    best_accuracy = 0.0

    for cycle in range(epochs):

        # ---------- training phase ----------
        net.train()
        cumulative_loss = 0.0

        for batch_x, batch_y in train_stream:

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            opt.zero_grad()

            logits = net(batch_x)
            loss_value = loss_fn(logits, batch_y)

            loss_value.backward()
            opt.step()

            cumulative_loss += loss_value.item() * batch_x.size(0)

        train_loss = cumulative_loss / len(train_stream.dataset)

        # ---------- validation phase ----------
        net.eval()

        total_correct = 0
        total_seen = 0
        val_loss_accum = 0.0

        with torch.no_grad():

            for batch_x, batch_y in val_stream:

                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)

                logits = net(batch_x)

                loss_value = loss_fn(logits, batch_y)
                val_loss_accum += loss_value.item() * batch_x.size(0)

                predictions = torch.argmax(logits, dim=1)

                total_seen += batch_y.size(0)
                total_correct += (predictions == batch_y).sum().item()

        val_loss = val_loss_accum / len(val_stream.dataset)
        accuracy = 100.0 * total_correct / total_seen

        print(
            f"Epoch {cycle + 1}/{epochs} | "
            f"Train Loss: {train_loss:.4f} | "
            f"Val Loss: {val_loss:.4f} | "
            f"Val Acc: {accuracy:.2f}%"
        )

        # ---------- checkpoint ----------
        if accuracy > best_accuracy:

            best_accuracy = accuracy

            save_dir = os.path.join("resnet_models", dataset_tag)
            os.makedirs(save_dir, exist_ok=True)

            torch.save(
                net.state_dict(),
                os.path.join(save_dir, "resnet18_best.pth")
            )

    print("Training finished — Best validation accuracy:", best_accuracy)

In [ ]:
# =====================================================
# MODEL VALIDATION HELPER
# =====================================================

def run_model_validation(net,
                         data_stream,
                         loss_fn,
                         device="cpu"):

    net.to(device)
    net.eval()

    total_correct = 0
    total_samples = 0
    accumulated_loss = 0.0

    with torch.no_grad():

        for batch_x, batch_y in data_stream:

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            logits = net(batch_x)

            loss_value = loss_fn(logits, batch_y)
            accumulated_loss += loss_value.item() * batch_x.size(0)

            preds = torch.argmax(logits, dim=1)

            total_samples += batch_y.size(0)
            total_correct += (preds == batch_y).sum().item()

    accuracy = 100.0 * total_correct / total_samples
    mean_loss = accumulated_loss / total_samples

    return accuracy, mean_loss

In [ ]:
# =====================================================
# DEVICE SELECTION
# =====================================================

if torch.cuda.is_available():
    compute_target = "cuda"
elif torch.backends.mps.is_available():
    compute_target = "mps"
else:
    compute_target = "cpu"


# =====================================================
# CIFAR PIPELINE
# =====================================================

cifar_train_set, cifar_val_set, cifar_test_set = build_cifar_resnet_sets()

cifar_train_stream = DataLoader(
    cifar_train_set,
    batch_size=64,
    shuffle=True
)

cifar_val_stream = DataLoader(
    cifar_val_set,
    batch_size=64,
    shuffle=False
)

cifar_net = create_resnet_backbone(class_count=10)

cifar_opt = optim.Adam(
    cifar_net.parameters(),
    lr=1e-4
)

loss_fn = nn.CrossEntropyLoss()

run_resnet_fit(
    cifar_net,
    dataset_tag="cifar",
    train_stream=cifar_train_stream,
    val_stream=cifar_val_stream,
    opt=cifar_opt,
    loss_fn=loss_fn,
    epochs=10,
    device=compute_target
)


# =====================================================
# DOGS vs CATS PIPELINE
# =====================================================

dvc_train_set, dvc_val_set = build_catdog_resnet_sets()

dvc_train_stream = DataLoader(
    dvc_train_set,
    batch_size=32,
    shuffle=True
)

dvc_val_stream = DataLoader(
    dvc_val_set,
    batch_size=32,
    shuffle=False
)

dvc_net = create_resnet_backbone(class_count=2)

dvc_opt = optim.Adam(
    dvc_net.parameters(),
    lr=1e-4
)

loss_fn = nn.CrossEntropyLoss()

run_resnet_fit(
    dvc_net,
    dataset_tag="dvc",
    train_stream=dvc_train_stream,
    val_stream=dvc_val_stream,
    opt=dvc_opt,
    loss_fn=loss_fn,
    epochs=10,
    device=compute_target
)

/var/folders/mz/34p9h3ld08ldtfp6qbl62tmc0000gn/T/ipykernel_45658/1812573342.py:9: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  batch = pickle.load(f, encoding='bytes')
/var/folders/mz/34p9h3ld08ldtfp6qbl62tmc0000gn/T/ipykernel_45658/1812573342.py:15: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  batch = pickle.load(f, encoding='bytes')


Epoch 1/10, Train Loss: 1.3414, Val Loss: 1.1672, Val Acc: 59.29%
Epoch 2/10, Train Loss: 0.9058, Val Loss: 0.9468, Val Acc: 66.90%
Epoch 3/10, Train Loss: 0.6853, Val Loss: 0.8507, Val Acc: 71.48%
Epoch 4/10, Train Loss: 0.5081, Val Loss: 0.7650, Val Acc: 73.51%
Epoch 5/10, Train Loss: 0.3389, Val Loss: 0.9067, Val Acc: 71.15%
Epoch 6/10, Train Loss: 0.2014, Val Loss: 0.9584, Val Acc: 72.77%
Epoch 7/10, Train Loss: 0.1146, Val Loss: 0.9036, Val Acc: 74.18%
Epoch 8/10, Train Loss: 0.0775, Val Loss: 0.9485, Val Acc: 73.82%
Epoch 9/10, Train Loss: 0.0701, Val Loss: 1.0792, Val Acc: 72.28%
Epoch 10/10, Train Loss: 0.0618, Val Loss: 1.0741, Val Acc: 73.50%
Training complete. Best Val Accuracy: 74.18
Epoch 1/10, Train Loss: 0.5482, Val Loss: 0.4396, Val Acc: 79.44%
Epoch 2/10, Train Loss: 0.3881, Val Loss: 0.3817, Val Acc: 82.04%
Epoch 3/10, Train Loss: 0.2794, Val Loss: 0.2527, Val Acc: 89.32%
Epoch 4/10, Train Loss: 0.1943, Val Loss: 0.2450, Val Acc: 89.22%
Epoch 5/10, Train Loss: 0.1251,

In [ ]:
df = pd.DataFrame(pd.read_csv('experiment_results.csv'))
best_cifar = df[df.dataset == "Cifar-10"].nlargest(1, "accuracy")
best_dvc = df[df.dataset == "Dogs vs Cats"].nlargest(1, "accuracy")

best_cifar_model_path = f"models/cifar/model_{best_cifar.iloc[0]['activation']}_{best_cifar.iloc[0]['init']}_{best_cifar.iloc[0]['optimizer']}_best.pth"
best_dvc_model_path = f"models/dvc/model_{best_dvc.iloc[0]['activation']}_{best_dvc.iloc[0]['init']}_{best_dvc.iloc[0]['optimizer']}_best.pth"

print("Best CIFAR-10 model path:", best_cifar_model_path)
print("Best Dogs vs Cats model path:", best_dvc_model_path)

Best CIFAR-10 model path: models/cifar/model_leaky_relu_random_rmsprop_best.pth
Best Dogs vs Cats model path: models/dvc/model_leaky_relu_kaiming_adam_best.pth


In [ ]:
# =====================================================
# EVALUATION SUMMARY BUILDER
# =====================================================

loss_fn = nn.CrossEntropyLoss()
summary_rows = []


# -----------------------------------------------------
# CIFAR — ResNet evaluation
# -----------------------------------------------------

cifar_backbone = create_resnet_backbone(class_count=10)

cifar_weights = torch.load(
    "resnet_models/cifar/resnet18_best.pth",
    map_location=compute_target
)

cifar_backbone.load_state_dict(cifar_weights)

cifar_acc, cifar_loss = run_model_validation(
    cifar_backbone,
    cifar_val_stream,
    loss_fn,
    compute_target
)

summary_rows.append({
    "dataset": "CIFAR-10",
    "model": "ResNet-18",
    "accuracy": cifar_acc,
    "loss": cifar_loss
})


# CIFAR — best custom CNN result (from experiment table)

best_cfg_cifar = best_cifar.iloc[0]

summary_rows.append({
    "dataset": "CIFAR-10",
    "model": (
        f"{best_cfg_cifar['activation']}_"
        f"{best_cfg_cifar['init']}_"
        f"{best_cfg_cifar['optimizer']}"
    ),
    "accuracy": best_cfg_cifar["accuracy"],
    "loss": best_cfg_cifar["val_loss"]
})


# -----------------------------------------------------
# Dogs vs Cats — ResNet evaluation
# -----------------------------------------------------

dvc_backbone = create_resnet_backbone(class_count=2)

dvc_weights = torch.load(
    "resnet_models/dvc/resnet18_best.pth",
    map_location=compute_target
)

dvc_backbone.load_state_dict(dvc_weights)

dvc_acc, dvc_loss = run_model_validation(
    dvc_backbone,
    dvc_val_stream,
    loss_fn,
    compute_target
)

summary_rows.append({
    "dataset": "Dogs vs Cats",
    "model": "ResNet-18",
    "accuracy": dvc_acc,
    "loss": dvc_loss
})


# Dogs vs Cats — best custom CNN result

best_cfg_dvc = best_dvc.iloc[0]

summary_rows.append({
    "dataset": "Dogs vs Cats",
    "model": (
        f"{best_cfg_dvc['activation']}_"
        f"{best_cfg_dvc['init']}_"
        f"{best_cfg_dvc['optimizer']}"
    ),
    "accuracy": best_cfg_dvc["accuracy"],
    "loss": best_cfg_dvc["val_loss"]
})


# -----------------------------------------------------
# EXPORT SUMMARY
# -----------------------------------------------------

comparison_table = pd.DataFrame(summary_rows)
comparison_table.to_csv("comparison_results.csv", index=False)

print("Comparison saved to comparison_results.csv")


Comparison saved to comparison_results.csv


In [ ]:
df = pd.DataFrame(pd.read_csv('comparison_results.csv'))
df

,dataset,model,accuracy,loss
0,CIFAR-10,ResNet-18,74.18,0.903581
1,CIFAR-10,leaky_relu_random_rmsprop,84.03,0.463258
2,Dogs vs Cats,ResNet-18,90.22,0.255094
3,Dogs vs Cats,leaky_relu_kaiming_adam,89.74,0.234952
